In [8]:

import ultralytics
from ultralytics import YOLO
import cv2
from sort.sort import *
from utils import get_car,get_license,write_csv,new_write


In [9]:
mot_tracker = Sort()

In [10]:
model = YOLO('yolov8n.pt')

In [11]:
license_model = YOLO('runs/detect/train2/weights/best.pt')

In [12]:
results ={}

In [17]:
cap = cv2.VideoCapture('demo.mp4')
#only need the car bike etc
vehicle =[2,3,5,7] 
detection_ = []
ret = True
frame_nmr =-1
while ret:
    frame_nmr+=1
    ret,frame = cap.read()
    if  ret and frame_nmr < 10:
        results[frame_nmr] = {} 
        
        detections = model(frame)[0]
        #will print x,y,x2,y2 ,confedence and classid 
        for detection in detections.boxes.data.tolist():
            x1,y1,x2,y2,score,class_id = detection 
            if int(class_id) in vehicle:
                detection_.append([x1,y1,x2,y2,score])
       
       
        #track vehicles
        #will have all the bounding box data and tracking id
        track_ids = mot_tracker.update(np.asarray(detection_))

        #now we need to detect licence plates
        licence_plates =  license_model(frame)[0]
        for licence_plate in licence_plates.boxes.data.tolist():
            x1,y1,x2,y2,score,class_id = licence_plate 

            #connect car to licence plate using cutom funtion
            xcar1,ycar1,xcar2,ycar2,car_id = get_car(licence_plate,track_ids)

            #crop the licence plate
            licence_plate_crop = frame[int(y1):int(y2),int(x1):int(x2)]

            #process the coped licence plates
            licence_plate_crop_gray = cv2.cvtColor(licence_plate_crop,cv2.COLOR_BGR2GRAY)
            _,licence_plate_crop_thresh = cv2.threshold(licence_plate_crop_gray,64,255,cv2.THRESH_BINARY_INV)

            # cv2.imshow('original',licence_plate_crop)
            # cv2.imshow('threshold',licence_plate_crop_thresh)
            # cv2.waitKey(0)

            #read license plate number
            license_plate_text,license_plate_text_score = get_license(licence_plate_crop_thresh)

            if license_plate_text is not None:
                results[frame_nmr][car_id] = {'car':{'bbox':[xcar1,ycar1,xcar2,ycar2]},
                                             'licence_plate':{'bbox':[x1,y1,x2,y2],
                                                              'text':license_plate_text,
                                                              'bbox_score':score,
                                                              'text_score':license_plate_text_score}}
                



new_write(results,'./test.csv')                
                
            
            



                
        
        



0: 416x640 2 cars, 2.2ms
Speed: 15.5ms preprocess, 2.2ms inference, 0.6ms postprocess per image at shape (1, 3, 416, 640)



0: 416x640 1 LicensePlate, 2.4ms
Speed: 1.2ms preprocess, 2.4ms inference, 0.5ms postprocess per image at shape (1, 3, 416, 640)

0: 416x640 2 cars, 2.3ms
Speed: 0.8ms preprocess, 2.3ms inference, 0.9ms postprocess per image at shape (1, 3, 416, 640)

0: 416x640 1 LicensePlate, 2.0ms
Speed: 0.9ms preprocess, 2.0ms inference, 0.5ms postprocess per image at shape (1, 3, 416, 640)

0: 416x640 2 cars, 2.0ms
Speed: 1.1ms preprocess, 2.0ms inference, 0.5ms postprocess per image at shape (1, 3, 416, 640)

0: 416x640 1 LicensePlate, 1.9ms
Speed: 0.8ms preprocess, 1.9ms inference, 0.8ms postprocess per image at shape (1, 3, 416, 640)

0: 416x640 2 cars, 1.9ms
Speed: 0.8ms preprocess, 1.9ms inference, 0.4ms postprocess per image at shape (1, 3, 416, 640)

0: 416x640 1 LicensePlate, 2.4ms
Speed: 1.0ms preprocess, 2.4ms inference, 0.5ms postprocess per image at shape (1, 3, 416, 640)

0: 416x640 2 cars, 1.9ms
Speed: 0.8ms preprocess, 1.9ms inference, 0.4ms postprocess per image at shape (1, 3, 416